In [1]:
%run stdPackages.ipynb

# Read database

In [2]:
from pyDbs import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables','Regulation'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}
db = read.dbFromWB(os.path.join(d['data'],'mGFTradeLarge.xlsx'), kwargs)
readSets(db)

This is very difficult for the solver (we should consider setting upper bounds on each plant instead of grouping plants - this should simplify things)

In [3]:
import time
times = {}
t0 = time.time()
# Initialize model:
m = mGFTrade.mSimple(db) # initialize using the database 'db' from above.
times['setup'] = time.time()-t0
# Initialize blocks:
m.preSolve()
m.initBlocks()
times['initBlocks'] = time.time()-t0-sum(times.values())
# Compile model:
m.blocks.compileParameters()
times['compileParameters'] = time.time()-t0-sum(times.values())
m.blocks.settingsFromCompiled()
times['settingsFromCompiled'] = time.time()-t0-sum(times.values())
m.blocks.inferGlobalDomains()
times['inferGlobalDomains'] = time.time()-t0-sum(times.values())
m.blocks.getDenseArgs()
times['getDenseArgs'] = time.time()-t0-sum(times.values())
args = m.blocks.lp_args
times['lp_args'] = time.time()-t0-sum(times.values())
sol = optimize.linprog(method = m.method, **args)
times['sol'] = time.time()-t0-sum(times.values())
m.postSolve(sol)
times['postSolve'] = time.time()-t0-sum(times.values())
times

{'setup': 0.18717479705810547,
 'initBlocks': 1.9276671409606934,
 'compileParameters': 3.1577529907226562,
 'settingsFromCompiled': 0.0,
 'inferGlobalDomains': 0.5752267837524414,
 'getDenseArgs': 2.414843797683716,
 'lp_args': 1.5613141059875488,
 'sol': 133.14674639701843,
 'postSolve': 1.9840750694274902}